In [1]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import math
import os
import time

In [2]:


mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
# Lista para armazenar os pontos desenhados
trail_points = []

MAX_DISTANCE = 50
tempo_inicial = time.time()


In [3]:
# Caminho do vídeo de entrada
video_path = 'Hands.mp4' #vídeo
output_path = 'HandsResultado.mp4'
# Verifica se o vídeo existe
if not os.path.isfile(video_path):
    raise FileNotFoundError(f"Arquivo de vídeo '{video_path}' não encontrado.")

# Usa o vídeo como entrada
cap = cv2.VideoCapture(video_path)

In [4]:
# Configura o vídeo de saída
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) 
frame_rate = int(cap.get(cv2.CAP_PROP_FPS)) 

out = cv2.VideoWriter(output_path, fourcc, frame_rate, (frame_width, frame_height))

In [6]:
cap = cv2.VideoCapture(0)
stroke_circles = []
strokes_time = []
stroke_time_init = time.time()
stroke = False;
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():#enquanto a camera estiver aberta
        ret, frame = cap.read() #resultados da captura da camera
        if ret==False:
            print('fim')
            break
        frame = cv2.resize(frame, (frame_width, frame_height))
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)#OpenCV usa o formato de cores BGR, mas o MediaPipe usa RGB
        image.flags.writeable = False #setFlag

        #Detections
        results = hands.process(image)#Envia a imagem para o modelo do MediaPipe para detectar mãos.
        
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)#Retornando a imagem para BGR
        if time.time() - tempo_inicial > 20:
            trail_points.clear()  # Apaga o desenho
            tempo_inicial = time.time()  # Reinicia o contador

        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand,mp_hands.HAND_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(0, 250, 0), thickness=2, circle_radius=3),
                                        mp_drawing.DrawingSpec(color=(0, 0, 250), thickness=2, circle_radius=1),
                                        )
            for hand_landmarks in results.multi_hand_landmarks:
                h, w, _ = frame.shape  # Altura e largura do frame

                # Coordenadas do polegar (landmark 4)
                x4 = int(hand_landmarks.landmark[4].x * w)
                y4 = int(hand_landmarks.landmark[4].y * h)

                # Coordenadas do indicador (landmark 8)
                x8 = int(hand_landmarks.landmark[8].x * w)
                y8 = int(hand_landmarks.landmark[8].y * h)
                
                distance = math.sqrt((x8 - x4) ** 2 + (y8 - y4) ** 2)
                # Se os pontos estiverem próximos, desenha no meio
                if distance < MAX_DISTANCE:
                    x_middle = (x4 + x8) // 2
                    y_middle = (y4 + y8) // 2

                    # Adiciona o ponto à trilha
                    trail_points.append((x_middle, y_middle))
                    

                # Desenha a trilha no vídeo
                for i in range(1, len(trail_points)):
                    cv2.line(image, trail_points[i - 1], trail_points[i], (0, 255, 0), 3)
                    if len(trail_points) > 30:
                        dx = abs(trail_points[i][0] - trail_points[i - 30][0])
                        dy = abs(trail_points[i][1] - trail_points[i - 30][1])
                        if dx < 2 and dy < 2 and stroke == False:
                            print(f"dx:{dx}dy:{dy}")
                            stroke_circles.append(trail_points[i])
                            stroke = True
                            stroke_time_init = time.time()
                        elif dx > 5 and dy > 5 and stroke == True:
                            stroke = False
                            strokes_time.append(time.time() - stroke_time_init)
                    
                # Desenha um círculo no último ponto detectado
                if len(trail_points) > 0:
                    cv2.circle(image, trail_points[-1], 5, (0, 0, 255), -1)
                # Desenha um círculo nos pontos de strokes                   
                for point in stroke_circles:
                    cv2.circle(image, point, 5, (255, 0, 255), -1)

        
        out.write(image)
        cv2.imshow('Hand Tracking', image)#mostrando a imagem
        

        if cv2.waitKey(10) & 0xFF == ord('q'):#apertar q para parar
            break

cap.release()
out.release()
cv2.destroyAllWindows()
print(f'Número de Pausas: {len(stroke_circles)}')
print(f'Média de tempo das Pausas: {sum(strokes_time)/len(strokes_time)}')
print(f"Vídeo com rastreamento salvo em: {output_path}")

dx:1dy:1
dx:1dy:1
dx:0dy:1
dx:1dy:1
dx:1dy:1
dx:1dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:0dy:1
dx:1dy:1
dx:1dy:1
dx:1dy:1
dx:1dy:1
Número de Pausas: 56
Média de tempo das Pausas: 0.008856053863252913
Vídeo com rastreamento salvo em: HandsResultado.mp4


In [15]:
mp_hands.HAND_CONNECTIONS

frozenset({(0, 1),
           (0, 5),
           (0, 17),
           (1, 2),
           (2, 3),
           (3, 4),
           (5, 6),
           (5, 9),
           (6, 7),
           (7, 8),
           (9, 10),
           (9, 13),
           (10, 11),
           (11, 12),
           (13, 14),
           (13, 17),
           (14, 15),
           (15, 16),
           (17, 18),
           (18, 19),
           (19, 20)})

![HandLandmarks](HandLandmarks.png)


In [4]:
mp_drawing.DrawingSpec??

Init signature:
mp_drawing.DrawingSpec(
    color: Tuple[int, int, int] = (224, 224, 224),
    thickness: int = 2,
    circle_radius: int = 2,
) -> None
Docstring:      DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2)
Source:        
@dataclasses.dataclass
class DrawingSpec:
  # Color for drawing the annotation. Default to the white color.
  color: Tuple[int, int, int] = WHITE_COLOR
  # Thickness for drawing the annotation. Default to 2 pixels.
  thickness: int = 2
  # Circle radius. Default to 2 pixels.
  circle_radius: int = 2
File:           e:\code\anaconda\envs\mediapipe_env\lib\site-packages\mediapipe\python\solutions\drawing_utils.py
Type:           type
Subclasses:     

In [ ]:
os.mkdir('Output Images')

In [16]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=1) as hands: 
    while cap.isOpened():
        ret, frame = cap.read()
        
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Detections
        print(results)
        
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=3),
                                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=1),
                                         )
            
        # Save our image    
        #cv2.imwrite(os.path.join('Output Images', '{}.jpg'.format(uuid.uuid1())), image)
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [13]:
mp_hands.HandLandmark.WRIST

<HandLandmark.WRIST: 0>

In [17]:
results.multi_hand_landmarks[0]

landmark {
  x: 0.362583309
  y: 0.786910057
  z: 7.17110936e-007
}
landmark {
  x: 0.466157675
  y: 0.757117629
  z: -0.0525502339
}
landmark {
  x: 0.555292487
  y: 0.681172252
  z: -0.0784128532
}
landmark {
  x: 0.614507496
  y: 0.603698969
  z: -0.100055844
}
landmark {
  x: 0.667273164
  y: 0.542636037
  z: -0.121519133
}
landmark {
  x: 0.493483216
  y: 0.494033098
  z: -0.0547132306
}
landmark {
  x: 0.532473862
  y: 0.369191796
  z: -0.089979127
}
landmark {
  x: 0.553037643
  y: 0.287244558
  z: -0.116770275
}
landmark {
  x: 0.567743421
  y: 0.212712616
  z: -0.136562735
}
landmark {
  x: 0.428782463
  y: 0.462227643
  z: -0.0552294478
}
landmark {
  x: 0.448530078
  y: 0.314434886
  z: -0.087371476
}
landmark {
  x: 0.461114824
  y: 0.218260646
  z: -0.113124691
}
landmark {
  x: 0.46920523
  y: 0.134387851
  z: -0.13214162
}
landmark {
  x: 0.365758419
  y: 0.46768716
  z: -0.0617025383
}
landmark {
  x: 0.369458884
  y: 0.330909461
  z: -0.0965648368
}
landmark {
  x: 0.3